In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
import sys
#sys.path.append("/content/drive/MyDrive/")
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/rhinoceros_hippopotamus_classification/


Mounted at /content/gdrive
/content/gdrive/MyDrive/rhinoceros_hippopotamus_classification


In [6]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 4.5 MB/s eta 0:00:00


In [7]:
from transformers import ViTForImageClassification,ViTConfig, ViTImageProcessor
import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader,TensorDataset
from torchvision import transforms
from tqdm import tqdm
from torchmetrics.functional import accuracy
import matplotlib.pyplot as plt
from pathlib import Path

from utils import load_dataset

In [8]:
save_path = Path("model.pch")
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')

# Freeze le model
for parameter in  model.parameters() :
  parameter.requires_grad = False

# Changer la derniere couche
nb_classe = 2
model.classifier = nn.Linear(model.config.hidden_size, nb_classe)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [10]:
train_images, train_labels = load_dataset(test=False, color=False)
test_images, test_labels = load_dataset(test=True, color=False)

transform_data = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
])

tensor_tr = torch.stack([transform_data(image) for image in train_images])
tensor_ts = torch.stack([transform_data(image) for image in test_images])


tensor_test = tensor_ts.expand(-1, 3, -1, -1).to(torch.float32)
tensor_train = tensor_tr.expand(-1, 3, -1, -1).to(torch.float32)

train_tensor = TensorDataset(tensor_train,torch.tensor(train_labels, dtype = torch.long))
test_tensor = TensorDataset(tensor_test,torch.tensor(test_labels, dtype = torch.long))

BATCH_SIZE = 64

Train = DataLoader(train_tensor,shuffle=True, batch_size = BATCH_SIZE)
Test = DataLoader(test_tensor,shuffle = False)

error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 1e-3)
device = torch.device('cuda' if torch.cuda.is_available()  else 'cpu')

In [ ]:
num_epochs = 10
model = model.to(device)
accuracy_train = []
loss_train = []

accuracy_test = []
loss_test = []

for epoch in range(num_epochs):
  acc_accuracy_train = 0
  acc_accuracy_test = 0
  acc_loss_train = 0
  acc_loss_test = 0
  data_train_tqdm = tqdm(Train,desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False, dynamic_ncols=True)

  for data,label in data_train_tqdm:
      # Forward pass
      data = data.to(device)
      label = label.to(device)
      outputs = model(data)
      prediction = outputs.logits
      predict_final = torch.argmax(prediction, axis = 1)
      acc_accuracy_train += accuracy(predict_final,label, task = "BINARY")
      loss_ = loss(prediction, label)
      acc_loss_train += loss_.item()
      optimizer.zero_grad()
      loss_.backward()
      optimizer.step()

  if epoch %  3 == 0 :
    with save_path.open('wb') as fp :
      torch.save(model,fp)

  acc = (acc_accuracy_train / len(Train))
  accuracy_train.append(acc)
  loss_train.append(acc_loss_train)


  with torch.no_grad() :
    for data_test,label_test in Test :
      data_test = data_test.to(device)
      label_test = label_test.to(device)
      output = model(data_test)
      prediction = output.logits
      acc_loss_test += loss(prediction,label_test).item()
      acc_accuracy_test += accuracy(torch.argmax(prediction,axis = 1),label_test,task = "BINARY")

  accuracy_test.append(acc_accuracy_test/len(Test))
  loss_test.append(acc_loss_test)

In [ ]:
list_acc_train = [acc.item() for acc in accuracy_train]
list_acc_test = [acc.item() for acc in accuracy_test]

In [ ]:
plt.figure(figsize=(12, 6))
epochs = range(1,num_epochs+1)

plt.title("Courbe des loss function en fonction de l'époque (B&W)")
plt.xlabel('Epoque')
plt.ylabel("Valeur de la loss")
plt.plot(epochs,loss_train, label = "train loss")
plt.plot(epochs,loss_test, label = "test loss")
plt.legend()


In [ ]:
plt.figure(figsize=(12, 6))
epochs = range(1,num_epochs+1)

plt.title("Courbe de l'accuracy en fonction de l'époque (Color)")
plt.xlabel('Epoque')
plt.ylabel("Valeur de l'accuracy")

plt.plot(epochs,list_acc_train,label = "accuracy train")
plt.plot(epochs,list_acc_test,label = "accuracy test")
plt.legend()
